In [1]:
from json import load
from pathlib import Path
import pandas as pd

metrics = load(Path("metrics.json").open())
devices = load(Path("devices.json").open())
sensors = load(Path("sensors.json").open())

In [2]:
def rename_sensor(x):
    if x in sensors["sensor2id_map"]:
        return sensors["sensor2id_map"][x].replace("-", "_")
    else:
        return x
    
def get_parent(s):
    if s.asset in sensors["sensor2rack_map"]:
        return sensors["sensor2rack_map"][s.asset]
    elif s.asset_type == "PDU - Rackmount":
        return s.asset.split("-")[1]
    else:
        return None
    
mdf = pd.DataFrame(metrics)
ddf = pd.DataFrame(devices).transpose().reset_index().rename(columns=dict(index="dev_id", type="dev_type", name="dev_name"))
df = mdf.merge(ddf[["dev_id", "dev_type"]], on="dev_id")

df2 = df.rename(columns=dict(dev_name="asset", md_name="data_point", dev_type="asset_type"))[["data_point", "asset", "asset_type", "value"]]
df2["parent_asset"] = df2.apply(get_parent, axis=1)
df2["asset"] = df2["asset"].apply(rename_sensor)

racks = df2[~df2.parent_asset.isna()].parent_asset.unique()
rack_df = pd.Series(racks).to_frame("asset")
rack_df["asset_type"] = "Rack"

df2 = pd.concat([df2, rack_df], axis=0)

df2.to_csv("data_list.csv", index=False)

In [3]:
parent_dict = df2.groupby("asset")["parent_asset"].first().to_dict()
type_dict = df2.groupby("asset")["asset_type"].first().to_dict()
dp_dict = df2.groupby("asset").data_point.unique().to_dict()

In [4]:
def get_path(asset, parent_dict, type_dict):
    component = f"{type_dict[asset]}.{asset}"
    parent = parent_dict.get(asset, None)
    if parent:
        return f"{get_path(parent, parent_dict, type_dict)}.{component}"
    else:
        return component

df2["path"] = df2.asset.apply(lambda x: get_path(x, parent_dict, type_dict))

In [6]:
df2["parent_type"] = df2.parent_asset.apply(lambda x: type_dict.get(x, None))
cc = df2.groupby(["parent_type", "asset_type", "parent_asset"]).asset.nunique().unstack("parent_asset").fillna(0).astype(int)
count_info = dict(min=cc.min(axis=1).to_dict(), max=cc.max(axis=1).to_dict())

In [8]:
def insert_into_schema(path, schema, parent_type=None):
    if isinstance(path, str):
        path = path.split(".")
    if path:
        asset_type = path.pop(0)
        if asset_type not in schema:
            schema[asset_type] = {"$type": asset_type, "$number": 0}
        asset = path.pop(0)
        
        if (parent_type, asset_type) in count_info["min"]:
            min_num = count_info["min"][(parent_type, asset_type)]
            max_num = count_info["max"][(parent_type, asset_type)]
            if min_num == max_num:
                schema[asset_type]["$number"] = min_num
            else:
                schema[asset_type]["$number"] = f"{min_num}-{max_num}"
        else:
            schema[asset_type]["$number"] += 1
        insert_into_schema(path, schema[asset_type], asset_type)
        
def insert_into_uses(path, data_point, uses):
    if isinstance(path, str):
        path = path.split(".")
    if path:
        asset_type = path.pop(0)
        asset = path.pop(0)
        if asset_type not in uses:
            uses[asset_type] = {}
        insert_into_uses(path, data_point, uses[asset_type])
    else:
        uses[data_point] = None
        

def post_process_uses(uses):
    res = []
    for k in sorted(uses.keys()):
        v = uses[k]
        if v:
            v = post_process_uses(v)
            res.append({k:v})
        else:
            res.append(k)
    return res
    
    

schema = {}
uses = {}
for path in df2.path.unique():
    insert_into_schema(path, schema)
    
for row in df2[~df2.data_point.isna()].itertuples():
    insert_into_uses(row.path, row.data_point.strip(), uses)

from rich import print
import yaml
    
# print(yaml.dump(schema))
# print(yaml.dump(post_process_uses(uses)))

In [22]:
df2

,data_point,asset,asset_type,value,parent_asset,path,parent_type
0,IPDU_Power_Outlet_08,BTCC-BA04-PDUR,PDU - Rackmount,70.0,BA04,Rack.BA04.PDU - Rackmount.BTCC-BA04-PDUR,Rack
1,Active Power,BTCC-BA04-PDUR,PDU - Rackmount,70.0,BA04,Rack.BA04.PDU - Rackmount.BTCC-BA04-PDUR,Rack
2,IPDU_Power_Outlet_03,BTCC-BA05-PDUL,PDU - Rackmount,49.0,BA05,Rack.BA05.PDU - Rackmount.BTCC-BA05-PDUL,Rack
3,IPDU_Power_Outlet_02,BTCC-BA05-PDUL,PDU - Rackmount,53.0,BA05,Rack.BA05.PDU - Rackmount.BTCC-BA05-PDUL,Rack
4,IPDU_Power_Outlet_08,BTCC-BA05-PDUL,PDU - Rackmount,66.0,BA05,Rack.BA05.PDU - Rackmount.BTCC-BA05-PDUL,Rack
...,...,...,...,...,...,...,...
219,NaN,AL01,Rack,NaN,NaN,Rack.AL01,None
220,NaN,AL05,Rack,NaN,NaN,Rack.AL05,None
221,NaN,AO03,Rack,NaN,NaN,Rack.AO03,None
222,NaN,AO04,Rack,NaN,NaN,Rack.AO04,None


In [17]:
df2.groupby(["asset", "data_point"]).value.agg(["min", "max"])

min    max
asset           data_point                        
BTCC-BA04-PDUR  Active Power           70.0   70.0
                IPDU_Power_Outlet_08   70.0   70.0
BTCC-BA05-PDUL  Active Power          223.0  223.0
                IPDU_Power_Outlet_02   53.0   53.0
                IPDU_Power_Outlet_03   49.0   49.0
...                                     ...    ...
pptb1-QH04-PDUR IPDU_Power_Outlet_06  145.0  145.0
                IPDU_Power_Outlet_07  126.0  126.0
                IPDU_Power_Outlet_08  120.0  120.0
                IPDU_Power_Outlet_09  159.0  159.0
                IPDU_Power_Outlet_10   67.0   67.0

[3390 rows x 2 columns]